In [1]:
import pandas as pd
import json

In [2]:
wandb_res = pd.read_csv('../log/wandb_res.csv')
wandb_res = wandb_res[~wandb_res['args.model_id'].isnull()][['Name', 'Tags', 'args.model_id', 'movingAverage', 'Training reward', 'Runtime']]

In [3]:
df = pd.DataFrame(json.load(open('../log/log.json'))).T.reset_index()
df = df.merge(wandb_res, left_on = 'index', right_on='args.model_id')

In [4]:
df_cnn = df[df.rnn_hidden.isnull()]\
.groupby(['num_revisit', 'Tags']).apply(lambda x: x.sort_values('best_reward', ascending=False)\
                              .iloc[:2, :]).reset_index(drop=True)
df_cnn = df_cnn[df_cnn.num_revisit.isin([1,4,12])]

In [5]:
df_cnn_rnn = df[~df.rnn_hidden.isnull()]\
.groupby(['num_revisit', 'Tags']).apply(lambda x: x.sort_values('best_reward', ascending=False)\
                              .iloc[:2, :]).reset_index(drop=True)

In [6]:
cnn_models =  df_cnn[['model_id', 'dropout']].set_index('model_id').to_dict()['dropout']
cnn_rnn_models = df_cnn_rnn.model_id.to_list()

In [8]:
df_cnn.to_csv('seq2seq_results.csv', index=False)

In [9]:
df_cnn_rnn.to_csv('attention_results.csv', index=False)

In [10]:
import json
with open('./seq2seq.json', 'w')as f:
    json.dump(cnn_models, f)

In [11]:
with open('./attention.json', 'w')as f:
    json.dump(cnn_rnn_models, f)